In [1]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

using Random

 Activating environment at `~/codes/hypergraph_modularities_code/Project.toml`


In [2]:
identity(p::Vector{Int64}) = p

function sum_of_ext_degs(p::Vector{Int64})
    soed = length(p) - 1
    return (sum(p), soed)
end

function all_or_nothing(p::Vector{Int64})
    is_aon = length(p) == 1
    return (sum(p), is_aon)
end

function rainbow(p::Vector{Int64})
    is_rainbow = length(p) == sum(p) && length(p) > 1
    return (sum(p), is_rainbow)
end

function all_or_nothing2(p::Vector{Int64})
    return length(p) == 1
end

all_or_nothing2 (generic function with 1 method)

In [3]:
function ll_ratios(Hyp, Z, agg, num_trials::Int64=100)
    function MLE_ll(Z1)
        Ω̂ = estimateΩEmpirically(Hyp, Z1; min_val=0, aggregator=agg)
        return Float64(sum(L(Hyp, Z1, Ω̂; α=0, bigInt=true)))    
    end
    data_ll = MLE_ll(Z)
    rand_ll = Float64[]
    for _ = 1:num_trials
        push!(rand_ll, MLE_ll(shuffle(Z)))
    end
    
    return data_ll, rand_ll
end

ll_ratios (generic function with 2 methods)

In [ ]:
#H, labels = read_hypergraph_data("contact-primary-school", 10)
#data_ll, rand_ll = ll_ratios(H, labels, all_or_nothing2, 500)
#data_ll .- rand_ll

In [ ]:
#H, labels = read_hypergraph_data("walmart-trips", 10)
#data_ll, rand_ll = ll_ratios(H, labels, all_or_nothing2, 100)
#data_ll .- rand_ll

In [ ]:
# Are cities or countries better communities for TrivagoClickout ?

for k = 3:15
    H1, labels1 = read_hypergraph_data("TrivagoClickout", k)
    Ω̂ = estimateΩEmpirically(H1, labels1; min_val=0, 
                              aggregator=all_or_nothing)
    ll1 = sum(L(H1, labels1, Ω̂; α=0, bigInt=true))

    H2, labels2 = read_hypergraph_data("TrivagoClickout-cities", k)
    Ω̂ = estimateΩEmpirically(H2, labels2; min_val=0, 
                              aggregator=all_or_nothing)
    ll2 = sum(L(H2, labels2, Ω̂; α=0, bigInt=true))

    diff = Float64(ll1 - ll2)
    println("$k $diff")
end

In [4]:
function MLE_ll(Hyp, Z, agg)
    Ω̂ = estimateΩEmpirically(Hyp, Z; min_val=0, aggregator=agg)
    return Float64(sum(L(Hyp, Z, Ω̂; α=0, bigInt=true)))    
end

function dyadic_MLE_ll(Hyp, Z, weighted::Bool)
    ω_in, ω_out = 
        computeDyadicResolutionParameter(Hyp, Z; mode="ω", weighted=weighted)
    return Float64(dyadicLogLikelihood(Hyp, Z, ω_in, ω_out, weighted))
end

dyadic_MLE_ll (generic function with 1 method)

In [5]:
for k = 3:6
    H, labels1 = read_hypergraph_data("TrivagoClickout", k)
    _, labels2 = read_hypergraph_data("TrivagoClickout-cities", k)


    println("max size = $k...")
    println("Polyadic MLE LL (countries): ", 
            MLE_ll(H, labels1, identity))
    println("Polyadic MLE LL (cities): ", 
            MLE_ll(H, labels2, identity))
    println("")

    println("Weighted Dyadic MLE LL (countries): ",     
            dyadic_MLE_ll(H, labels1, true))
    println("Weighted Dyadic MLE LL (cities): ",     
            dyadic_MLE_ll(H, labels2, true))
    println("")

    println("Unweighted Dyadic MLE LL (countries): ",     
            dyadic_MLE_ll(H, labels1, false))
    println("Unweighted Dyadic MLE LL (cities): ",     
            dyadic_MLE_ll(H, labels2, false))    
    
    println("")
end

max size = 3...
Polyadic MLE LL (countries): -6.241708403589702e6
Polyadic MLE LL (cities): -2.680765341324251e11

Weighted Dyadic MLE LL (countries): 212822.75119605014
Weighted Dyadic MLE LL (cities): 380391.80863090965

Unweighted Dyadic MLE LL (countries): 425645.5023921003
Unweighted Dyadic MLE LL (cities): 760783.6172618193

max size = 4...
Polyadic MLE LL (countries): -1.5105591745731872e7
Polyadic MLE LL (cities): -5.825459731279067e11

Weighted Dyadic MLE LL (countries): 363120.37528979406
Weighted Dyadic MLE LL (cities): 648265.8475551737

Unweighted Dyadic MLE LL (countries): 876460.7118677769
Unweighted Dyadic MLE LL (cities): 1.5641536031144669e6

max size = 5...
Polyadic MLE LL (countries): -5.581173909750826e6
Polyadic MLE LL (cities): -4.1616365774670806e14

Weighted Dyadic MLE LL (countries): 458253.15205535025
Weighted Dyadic MLE LL (cities): 808483.3323049098

Unweighted Dyadic MLE LL (countries): 1.2566817560339351e6
Unweighted Dyadic MLE LL (cities): 2.204969344897